In [29]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

import os

In [30]:
# 1. Create a function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata= pd.read_csv(kaggle_file)
    ratings= pd.read_csv(ratings_file)

    # 3. Open the read the Wikipedia data JSON file.
    with open(wiki_file, mode='r') as file:
        wiki_movies_raw = json.load(file)
    
    # 4. Read in the raw wiki movie data as a Pandas DataFrame.
    wiki_movies_df= pd.DataFrame(wiki_movies_raw)
    
    # 5. Return the three DataFrames
    return wiki_movies_df, kaggle_metadata, ratings

# 6 Create the path to your file directory and variables for the three files. 
file_dir = 'C:\\Users\Billy\Desktop\Bootcamp\Mods\Mod 8\Movies-ETL\Resources'

# Wikipedia data
wiki_file = os.path.join('Resources', 'wikipedia-movies.json')
# Kaggle metadata
kaggle_file = os.path.join('Resources', 'movies_metadata.csv')
# MovieLens rating data.
ratings_file = os.path.join('Resources', 'ratings.csv')

# 7. Set the three variables in Step 6 equal to the function created in Step 1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

C:\Users\Billy\AppData\Local\Temp/ipykernel_14340/359838319.py:30: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  wiki_file, kaggle_file, ratings_file = extract_transform_load()


In [31]:
# 8. Set the DataFrames from the return statement equal to the file names in Step 6. 
wiki_movies_df = wiki_file
kaggle_metadata = kaggle_file
ratings = ratings_file

In [ ]:
# 9. Check the wiki_movies_df DataFrame.
wiki_movies_df

In [ ]:
# 10. Check the kaggle_metadata DataFrame.
kaggle_metadata.head()

In [ ]:
# 11. Check the ratings DataFrame.
ratings.head()

In [ ]:
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))

In [ ]:
wiki_movies_df

In [ ]:
for index in wiki_movies_df:
    try:
        wiki_movies_df['imdb_link'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates('imdb_link', inplace= True)
    except Exception as ex:
        print(f'error {ex}')

In [ ]:
print(len(wiki_movies_df))

In [ ]:
wiki_movies_df

In [ ]:
print(len(wiki_movies_df))

In [ ]:
wiki_movies_df.columns

In [ ]:
wiki_columns= wiki_movies_df.isnull().sum()

In [ ]:
wiki_columns= pd.DataFrame(wiki_columns)

In [ ]:
[[column,wiki_movies_df[column].isnull().sum()] for column in wiki_movies_df.columns]

In [ ]:
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() == 0]

In [ ]:
len(wiki_columns_to_keep)

In [ ]:
box_office = wiki_movies_df['Box office'].dropna()

box_office

In [ ]:
box_office[box_office.map(lambda x: type(x) !=str)]

In [ ]:
box_office.dtypes

In [ ]:
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) !=str else x)

box_office[box_office.map(lambda x: type(x) !=str)]

In [ ]:
# Form 1 edit for regular expression ($123.4 millions and billions)

form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'

print(box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum())

# Form 2 edit for regular expression ($123,456,789)

form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'
print(box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum())

In [ ]:
def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan

    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
        value = float(s) * 10**6

        # return value
        return value

    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
        value = float(s) * 10**9

        # return value
        return value

    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
        s = re.sub('\$|,','', s)

        # convert to float
        value = float(s)

        # return value
        return value

    # otherwise, return NaN
    else:
        return np.nan

In [ ]:
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})',
                                                      flags=re.IGNORECASE)[0].apply(parse_dollars)

wiki_movies_df

In [ ]:
release_date = wiki_movies_df['Release date'].apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
# release date re forms

date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
date_form_two = r'\d{4}.[01\d.[01]\d.[0123]\d]'
date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
date_form_four = r'\d{4}'

In [ ]:
wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

In [ ]:
wiki_movies_df

In [ ]:
running_time = wiki_movies_df['Running time'].apply(lambda x: ' '.join(x) if type(x) == list else x)

running_time[running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False) != True]

running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')

running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)

wiki_movies_df